In [6]:
# Setup cell 1: clone the master's thesis GitHub repository if not already cloned

import os

repo_name = "masters_thesis"

if os.path.basename(os.getcwd()) != repo_name:
    # Check if the repository does not exist
    if not os.path.exists(repo_name):
        # Clone the repository if it doesn't exist
        print("Cloning the GitHub repository...")
        !git clone https://github.com/danimue/masters_thesis.git
        print("Successfully clonedGitHub repository.")
    else:
        print("Repository directory exists, but it may not be a valid Git repository.")

    # Change directory to the repository
    print("Changing directory to the repository...")
    %cd {repo_name}
    print("Current directory is now:", os.getcwd())
else:
    print("Already in the correct directory. Proceeding with the notebook.")

Already in the correct directory. Proceeding with the notebook.


In [ ]:
# Setup cell 2: only run to update changes from GitHub

repo_name = "masters_thesis"

if os.path.basename(os.getcwd()) == repo_name:
    print("Fetching latest changes from GitHub...")
    !git pull origin main
    print("Repository is now up to date.")
else:
    print("Did nothing.")

In [7]:
# use this to properly set everything up
# To-Do: check if this actually works
# !pip install -r requirements.txt

In [4]:
# this could be obsolete if the previous cell works properly!
!pip install vllm triton

from IPython.display import clear_output
clear_output(wait=False)

Current working directory: /home/daniel-mueller92/Projects/masters_thesis
Script location: /home/daniel-mueller92/Projects/masters_thesis/utils.py
ARC data complete.


In [8]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=8000)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

llm = LLM(model="Qwen/Qwen2.5-7B-Instruct-AWQ", quantization="awq")

# Figure out error handling here
# RuntimeError: Failed to infer device type
# This seems to happen if there is no GPU connected in Google Colab
# maybe get the pro subscription finally...

/home/daniel-mueller92/Projects/masters_thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-03 21:45:19 __init__.py:187] No platform detected, vLLM is running on UnspecifiedPlatform


2025-02-03 21:45:20,186	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [1]:
from utils import check_dataset, load_files_from_json, json_to_string, create_prompt

In [2]:
check_dataset()
train, eval = load_files_from_json()
prompt = create_prompt("0d3d703e", train, eval)
print(prompt)

Current working directory: /home/daniel-mueller92/Projects/masters_thesis
Script location: /home/daniel-mueller92/Projects/masters_thesis/utils.py
ARC data complete.
Here is are some examples of the transformation from `input` to `output`: 

Training Examples
Example 1: Input
[
[3, 1, 2],
[3, 1, 2],
[3, 1, 2]]

Example 1: Output
[
[4, 5, 6],
[4, 5, 6],
[4, 5, 6]]

Example 2: Input
[
[2, 3, 8],
[2, 3, 8],
[2, 3, 8]]

Example 2: Output
[
[6, 4, 9],
[6, 4, 9],
[6, 4, 9]]

Example 3: Input
[
[5, 8, 6],
[5, 8, 6],
[5, 8, 6]]

Example 3: Output
[
[1, 9, 2],
[1, 9, 2],
[1, 9, 2]]

Example 4: Input
[
[9, 4, 2],
[9, 4, 2],
[9, 4, 2]]

Example 4: Output
[
[8, 3, 6],
[8, 3, 6],
[8, 3, 6]]



You'll need to carefully reason in order to determine the transformation rule. Start your response by carefully reasoning in <reasoning></reasoning> tags. Then, implement the transformation in code.

For your reasoning, please pay close attention to where the objects in the grid are in the input and where the

In [9]:
import base64
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
# Path to your image
image_path = "/home/daniel-mueller92/Pictures/Screenshots/Screenshot From 2025-02-17 15-41-25.png"

# Getting the Base64 string
base64_image = encode_image(image_path)

# completion = instead of stream
stream = client.chat.completions.create(
    extra_body={},
    model="qwen/qwen2.5-vl-72b-instruct:free",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "This image shows transformations from input to output. Based on the examples you see, can you deduce what the underlying transformation rule is?"},
                {
                    "type": "image_url",
                    #"image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                }
            ]
        }
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

#print(completion.choices[0].message.content)

The transformation rule appears to be as follows:

1. **Color Change**: Each gray bar in the input is transformed into a colored bar in the output. The colors are assigned in a specific order: red, green, blue, and yellow.

2. **Order of Colors**: The colors are assigned in a sequential manner based on the position of the bars from left to right. The first bar becomes red, the second becomes green, the third becomes blue, and the fourth becomes yellow. If there are more than four bars, the sequence repeats.

3. **Height Preservation**: The height of each bar is preserved from the input to the output.

Let's break down the examples:

- **Example 1**:
  - Input: Four bars of different heights.
  - Output: The first bar is red, the second is green, the third is blue, and the fourth is yellow, with their respective heights preserved.

- **Example 2**:
  - Input: Four bars of different heights.
  - Output: The first bar is yellow, the second is red, the third is green, and the fourth is blu